In [ ]:
from os import listdir
import numpy as np
from tqdm import tqdm
from os.path import join
import cv2
import matplotlib.pyplot as plt



CAT = ["circle","square","star","triangle"]
IMG_DATA = []
X = []
Y = []
DATA_DIR = "./Documents/shapes/shapes"

In [ ]:
def preprocess():
    for cat in (CAT):
        for img in tqdm(listdir(join(DATA_DIR,cat))):
            imgData=cv2.resize(cv2.imread(join(DATA_DIR,cat,img),cv2.IMREAD_GRAYSCALE),(100,100))
            IMG_DATA.append([imgData,cat])
            
preprocess()

In [ ]:
from random import shuffle
shuffle(IMG_DATA)
for feature, label in IMG_DATA:
    X.append(feature)
    Y.append(label)

In [ ]:
for i in range(len(Y)):
    Y[i]=CAT.index(Y[i])

In [ ]:
X=np.array(X)
Y=np.array(Y)
X=X.reshape(-1,100,100,1)

In [ ]:
X=X/np.float32(255)

In [ ]:
g = np.split(X, 2)
X_train = g[0]
X_test = g[1]

h = np.split(Y, 2)
Y_train = h[0]
Y_test = h[1]

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPooling2D
#with tpu_strategy.scope():
model = Sequential()

model.add(Conv2D(64,(3,3),activation = "relu", input_shape = (100,100,1)))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(64,(3,3),activation = "relu" ))
model.add(MaxPooling2D((2,2)))

model.add(Flatten())
model.add(Dense(256, activation = 'sigmoid'))
model.add(Dropout(0.2))
model.add(Dense(256, activation='sigmoid'))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train,Y_train, batch_size = 256, epochs=20,validation_split=0.2)

In [ ]:
from tensorflow.keras.models import load_model

model.save("four-shape-model-10")

In [ ]:
# from tensorflow.keras.models import load_model

# import 

new = load_model("four-shape-model-10")

In [ ]:
testData = cv2.resize(cv2.imread("./Downloads/test_images/1200px-Star_full.svg.png",cv2.IMREAD_GRAYSCALE),(100,100))

plt.imshow(testData,cmap="gray")

plt.show()

testData=testData.reshape(1,100,100,1)

testData.shape

In [ ]:
# pred=new.predict(testData)
# print(CAT[np.argmax(pred)])
# # print(pred)

In [ ]:
test_loss, test_acc = model.evaluate(X_test,Y_test)


print("test_accuracy: ", test_acc) 